In [1]:
from mdagent import MDAgent

In [3]:
llm_var = "claude-3-opus-20240229"
prompt6 = "Simulate 1A3N and 7VDE (two PDB IDs matching hemoglobin) with identical parameters. Find the appropriate parameters for simulating hemoglobin from the literature. Then, plot the radius of gyration throughout both simulations."
paper_dir = "paper_collection"
tools = "all"
agent = MDAgent(agent_type="Structured", model=llm_var, top_k_tools=tools, paper_dir=paper_dir, use_memory=False)
print (llm_var)

claude-3-opus-20240229


In [4]:
agent.run(prompt6)

Thought: To simulate 1A3N and 7VDE with identical parameters and plot the radius of gyration, I will need to:

1. Download the PDB files for 1A3N and 7VDE
2. Find appropriate simulation parameters for hemoglobin in the literature 
3. Set up and run simulations for both PDB files using the parameters
4. Calculate and plot the radius of gyration for each trajectory

I will start by downloading the PDB files.

Action:
```
{
  "action": "PDBFileDownloader",
  "action_input": "1A3N"
}
```

PDB file found with this ID: 1A3N
Thought: Now that I have downloaded the PDB file for 1A3N, I will download the PDB file for 7VDE as well.

Action:
```
{
  "action": "PDBFileDownloader",
  "action_input": "7VDE"
}
```PDB file found with this ID: 7VDE
Thought: Now that I have downloaded the PDB files for both 1A3N and 7VDE, the next step is to search the literature to find appropriate simulation parameters for hemoglobin.

Action:
```
{
  "action": "LiteratureSearch",
  "action_input": "What are appropria

[18:20:38] Starting paper search for 'hemoglobin molecular dynamics simulation parameters, 2000-2024'.

           paper_search for query 'hemoglobin molecular dynamics simulation parameters, 2000-2024' returned 8      
           papers.

           Status: Paper Count=8 | Relevant Papers=0 | Current Evidence=0 | Current Cost=$0.0000

           Starting paper search for 'molecular dynamics simulation hemoglobin, 2015-2024'.

...

Error adding solvent ValueError – Found multiple NonbondedForce tags with different 1-4 scales
It looks like there was an error building the system due to an issue with the CHARMM36 forcefield file. The error message indicates there are multiple NonbondedForce tags with different 1-4 scales. To resolve this, I will try using a different forcefield.

Action:
```
{
  "action": "SetUpandRunFunction",
  "action_input": {
    "pdb_id": "1A3N_182204",
    "forcefield_files": ["amber14-all.xml"],
    "system_params": {
      "nonbondedMethod": "PME",
      "nonbondedCutoff": "1.2 * nanometers",
      "constraints": "HBonds",
      "rigidWater": true,
      "constraintTolerance": 0.00001,
      "solvate": true
    },
    "integrator_params": {
      "integrator_type": "LangevinMiddle",
      "Temperature": "300 * kelvin",
      "Friction": "1.0 / picoseconds",
      "Timestep": "0.002 * picoseconds",
      "Pressure": "1.0 * bar"
    },
    "simulation_params": {
      "Ensemble": "NPT",
     

Did not recognize residue CL; did you forget to call .add_molecules() to add it?


It looks like there was an issue with the amber14 forcefield not having parameters for the CL (chloride ion) residue in the 1A3N PDB file. To resolve this, I will remove the heterogens (including the chloride ions) when cleaning the PDB file, and increase the salt concentration in the solvent to compensate. Then I will retry the simulation setup.

Action:
```
{
  "action": "CleaningToolFunction",
  "action_input": {
    "pdb_id": "1A3N_182024",
    "replace_nonstandard_residues": true,
    "add_missing_atoms": true,
    "remove_heterogens": true,
    "remove_water": true,
    "add_hydrogens": true,
    "add_hydrogens_ph": 7.0
  }
}
```

file path: /Users/samanthacox/Desktop/md-agent/ckpt/ckpt_71/pdb/1A3N_raw_20241106_182024.pdb
Now that the 1A3N PDB file has been cleaned to remove heterogens, I will retry setting up and running the simulation with the amber14 forcefield and an increased salt concentration of 0.3 M/L to compensate for the removed chloride ions.

Action:
```
{
  "action"

Did not recognize residue CL; did you forget to call .add_molecules() to add it?


It looks like there are still issues with the amber14 forcefield not having parameters for the chloride ions, even after removing heterogens from the PDB file. Since the literature suggests using the CHARMM36 forcefield for hemoglobin simulations, I will try again with that forcefield but include the charmm36 water model file as well to avoid the previous error with multiple NonbondedForce tags.

Action:
```
{
  "action": "SetUpandRunFunction",
  "action_input": {
    "pdb_id": "1A3N_182335",
    "forcefield_files": ["charmm36.xml", "charmm36/water.xml"],
    "system_params": {
      "nonbondedMethod": "PME",
      "nonbondedCutoff": "1.2 * nanometers",
      "constraints": "HBonds",
      "rigidWater": true,
      "constraintTolerance": 0.00001,
      "solvate": true
    },
    "integrator_params": {
      "integrator_type": "LangevinMiddle",
      "Temperature": "300 * kelvin",
      "Friction": "1.0 / picoseconds",
      "Timestep": "0.002 * picoseconds",
      "Pressure": "1.0 * ba

#run was cancelled after several days of failing